This notebook provides examples to go along with the [textbook](https://underactuated.csail.mit.edu/trajopt.html).  I recommend having both windows open, side-by-side!


In [2]:
import numpy as np
from IPython.display import HTML, display
from matplotlib import pyplot as plt
from pydrake.all import (
    ConnectPlanarSceneGraphVisualizer,
    DiagramBuilder,
    DirectCollocation,
    FiniteHorizonLinearQuadraticRegulator,
    FiniteHorizonLinearQuadraticRegulatorOptions,
    LogVectorOutput,
    MakeFiniteHorizonLinearQuadraticRegulator,
    MultibodyPlant,
    Parser,
    PiecewisePolynomial,
    PlanarSceneGraphVisualizer,
    RobotDiagramBuilder,
    SceneGraph,
    Simulator,
    SnoptSolver,
    Solve,
    TrajectorySource,
)
from pydrake.examples import AcrobotGeometry, AcrobotPlant, PendulumPlant, PendulumState

from underactuated import ConfigureParser, running_as_notebook
from underactuated.pendulum import PendulumVisualizer
from underactuated.pyplot_utils import AdvanceToAndVisualize, AnimatePositionTrajectory

# Direct Collocation for the Pendulum

In [ ]:
def pend_dircol():
    plant = PendulumPlant()
    context = plant.CreateDefaultContext()

    N = 21
    max_dt = 0.5
    N * max_dt
    dircol = DirectCollocation(
        plant,
        context,
        num_time_samples=N,
        minimum_time_step=0.05,
        maximum_time_step=max_dt,
    )
    prog = dircol.prog()

    dircol.AddEqualTimeIntervalsConstraints()

    torque_limit = 3.0  # N*m.
    u = dircol.input()
    dircol.AddConstraintToAllKnotPoints(-torque_limit <= u[0])
    dircol.AddConstraintToAllKnotPoints(u[0] <= torque_limit)

    initial_state = PendulumState()
    initial_state.set_theta(0.0)
    initial_state.set_thetadot(0.0)
    prog.AddBoundingBoxConstraint(
        initial_state.get_value(),
        initial_state.get_value(),
        dircol.initial_state(),
    )
    # More elegant version is blocked on drake #8315:
    # dircol.AddLinearConstraint(
    #     dircol.initial_state() == initial_state.get_value())

    final_state = PendulumState()
    final_state.set_theta(np.pi)
    final_state.set_thetadot(0.0)
    prog.AddBoundingBoxConstraint(
        final_state.get_value(), final_state.get_value(), dircol.final_state()
    )
    # dircol.AddLinearConstraint(dircol.final_state() == final_state.get_value())

    R = 10  # Cost on input "effort".
    dircol.AddRunningCost(R * u[0] ** 2)

    initial_x_trajectory = PiecewisePolynomial.FirstOrderHold(
        [0.0, 4.0], [initial_state.get_value(), final_state.get_value()]
    )
    dircol.SetInitialTrajectory(PiecewisePolynomial(), initial_x_trajectory)

    result = Solve(prog)
    assert result.is_success()

    x_trajectory = dircol.ReconstructStateTrajectory(result)

    fig, ax = plt.subplots()

    x_knots = np.hstack(
        [
            x_trajectory.value(t)
            for t in np.linspace(
                x_trajectory.start_time(), x_trajectory.end_time(), 100
            )
        ]
    )

    ax.set_xlabel("$q$")
    ax.set_ylabel("$\dot{q}$")
    ax.plot(x_knots[0, :], x_knots[1, :])
    display(plt.show())

    # Animate the result.
    vis = PendulumVisualizer(show=False)
    ani = vis.animate(x_trajectory)
    display(HTML(ani.to_jshtml()))


pend_dircol()

# Direct Collocation for the Acrobot

Almost identical code works to swing-up the Acrobot.

In [ ]:
def dircol_acrobot():
    plant = AcrobotPlant()
    context = plant.CreateDefaultContext()

    dircol = DirectCollocation(
        plant,
        context,
        num_time_samples=21,
        minimum_time_step=0.05,
        maximum_time_step=0.2,
    )
    prog = dircol.prog()

    dircol.AddEqualTimeIntervalsConstraints()

    # Add input limits.
    torque_limit = 8.0  # N*m.
    u = dircol.input()
    dircol.AddConstraintToAllKnotPoints(-torque_limit <= u[0])
    dircol.AddConstraintToAllKnotPoints(u[0] <= torque_limit)

    initial_state = (0.0, 0.0, 0.0, 0.0)
    prog.AddBoundingBoxConstraint(initial_state, initial_state, dircol.initial_state())
    # More elegant version is blocked on drake #8315:
    # dircol.AddLinearConstraint(dircol.initial_state() == initial_state)

    final_state = (np.pi, 0.0, 0.0, 0.0)
    prog.AddBoundingBoxConstraint(final_state, final_state, dircol.final_state())
    # dircol.AddLinearConstraint(dircol.final_state() == final_state)

    R = 10  # Cost on input "effort".
    dircol.AddRunningCost(R * u[0] ** 2)

    # Add a final cost equal to the total duration.
    dircol.AddFinalCost(dircol.time())

    # Give an initial guess from a solution I like.
    initial_x_trajectory = PiecewisePolynomial.FirstOrderHold(
        [0.0, 0.2, 1.4, 2.0, 4.0],
        np.column_stack(
            (
                initial_state,
                [0.35, -0.71, 2.63, -5.34],
                [-1.19, 0.77, -6.87, 15.17],
                [1.12, 2.48, 5.99, -3.22],
                final_state,
            )
        ),
    )
    # This initial guess is enough on some platforms. (◡︵◡)
    # initial_x_trajectory = PiecewisePolynomial.FirstOrderHold(
    #    [0.0, 4.0], np.column_stack((initial_state, final_state))
    # )
    dircol.SetInitialTrajectory(PiecewisePolynomial(), initial_x_trajectory)

    solver = SnoptSolver()
    solver_id = solver.solver_id()
    major_tol = 1e-3
    minor_tol = 1e-3
    prog.SetSolverOption(solver_id, "Feasibility tolerance", major_tol)
    prog.SetSolverOption(solver_id, "Major feasibility tolerance", major_tol)
    prog.SetSolverOption(solver_id, "Major optimality tolerance", major_tol)
    prog.SetSolverOption(solver_id, "Minor feasibility tolerance", minor_tol)
    prog.SetSolverOption(solver_id, "Minor optimality tolerance", minor_tol)
    result = Solve(prog)
    assert result.is_success()

    u_trajectory = dircol.ReconstructInputTrajectory(result)
    times = np.linspace(u_trajectory.start_time(), u_trajectory.end_time(), 100)
    u_values = u_trajectory.vector_values(times)

    plt.figure()
    plt.plot(times, u_values.T)
    plt.xlabel("time (seconds)")
    plt.ylabel("force (Newtons)")
    display(plt.show())

    x_trajectory = dircol.ReconstructStateTrajectory(result)
    x_values = x_trajectory.vector_values(times)

    plt.figure()
    plt.plot(times, x_values.T, label=["θ₁", "θ₂", "dθ₁dt", "dθ₂dt"])
    plt.xlabel("time (seconds)")
    plt.legend()
    display(plt.show())

    builder = DiagramBuilder()
    source = builder.AddSystem(TrajectorySource(x_trajectory))
    scene_graph = builder.AddSystem(SceneGraph())
    AcrobotGeometry.AddToBuilder(builder, source.get_output_port(0), scene_graph)
    visualizer = builder.AddSystem(
        PlanarSceneGraphVisualizer(
            scene_graph, xlim=[-4.0, 4.0], ylim=[-4.0, 4.0], show=False
        )
    )
    builder.Connect(scene_graph.get_query_output_port(), visualizer.get_input_port(0))
    simulator = Simulator(builder.Build())

    # Simulate and animate
    AdvanceToAndVisualize(
        simulator,
        visualizer,
        x_trajectory.end_time() if running_as_notebook else 0.1,
    )

    return x_trajectory, u_trajectory


x_trajectory, u_trajectory = dircol_acrobot()

## Trajectory stabilization with (finite-horizon) LQR

The visualization above was an animation of the solution trajectory, it was not a simulation!  In fact, if you simulate only the planned torques connected to the Acrobot, then you will see that the trajectory does not follow the planned state trajectory.  The trajectory optimization only satisfies the dynamic constraints to some loose tolerance, and the (open-loop) trajectory is unstable.

To do better, we must stabilizing the planned trajectory with feedback.  We do that here with a finite-horizon LQR design, which takes a time-varying linearization of the plant along the trajectory.

In [ ]:
def finite_horizon_lqr(x_trajectory, u_trajectory):
    options = FiniteHorizonLinearQuadraticRegulatorOptions()
    options.x0 = x_trajectory
    options.u0 = u_trajectory

    builder = DiagramBuilder()
    plant = builder.AddSystem(AcrobotPlant())
    context = plant.CreateDefaultContext()
    Q = np.diag([10.0, 10.0, 1.0, 1.0])
    options.Qf = Q
    regulator = builder.AddSystem(
        MakeFiniteHorizonLinearQuadraticRegulator(
            plant,
            context,
            t0=options.u0.start_time(),
            tf=options.u0.end_time(),
            Q=Q,
            R=np.eye(1),
            options=options,
        )
    )
    builder.Connect(regulator.get_output_port(0), plant.get_input_port(0))
    builder.Connect(plant.get_output_port(0), regulator.get_input_port(0))
    input_logger = LogVectorOutput(regulator.get_output_port(0), builder)
    state_logger = LogVectorOutput(plant.get_output_port(0), builder)

    scene_graph = builder.AddSystem(SceneGraph())
    AcrobotGeometry.AddToBuilder(builder, plant.get_output_port(0), scene_graph)
    visualizer = builder.AddSystem(
        PlanarSceneGraphVisualizer(
            scene_graph, xlim=[-4.0, 4.0], ylim=[-4.0, 4.0], show=False
        )
    )
    builder.Connect(scene_graph.get_query_output_port(), visualizer.get_input_port(0))
    diagram = builder.Build()
    simulator = Simulator(diagram)

    # Simulate and animate
    AdvanceToAndVisualize(
        simulator,
        visualizer,
        options.u0.end_time() if running_as_notebook else 0.1,
    )

    fig, ax = plt.subplots(2, 1)
    ax[0].plot(
        options.u0.get_segment_times(),
        options.u0.vector_values(options.u0.get_segment_times()).T,
    )
    input_log = input_logger.FindLog(simulator.get_context())
    ax[0].plot(input_log.sample_times(), input_log.data().T)
    ax[0].legend(("u nominal", "u actual"))

    ax[1].plot(
        options.x0.get_segment_times(),
        options.x0.vector_values(options.x0.get_segment_times()).T,
        "b",
    )
    state_log = state_logger.FindLog(simulator.get_context())
    ax[1].plot(state_log.sample_times(), state_log.data().T, "g")
    ax[1].legend(("x nominal", "x actual"))
    display(plt.show())


finite_horizon_lqr(x_trajectory, u_trajectory)

# Direct Collocation for the Cart-Pole

While the previous two examples used equations of motion that we derived and typed in manually, for the Cart-Pole we will use the Drake dynamics engine and visualization via [`MultibodyPlant`](https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_multibody_plant.html) and [`SceneGraph`](https://drake.mit.edu/doxygen_cxx/classdrake_1_1geometry_1_1_scene_graph.html).

In [ ]:
def dircol_cartpole():
    builder = RobotDiagramBuilder(time_step=0.0)
    ConfigureParser(builder.parser())
    builder.parser().AddModelsFromUrl("package://underactuated/models/cartpole.urdf")
    diagram = builder.Build()

    context = diagram.CreateDefaultContext()
    dircol = DirectCollocation(
        diagram,
        context,
        num_time_samples=21,
        minimum_time_step=0.1,
        maximum_time_step=0.4,
        input_port_index=diagram.GetInputPort("plant_actuation").get_index(),
    )
    prog = dircol.prog()

    dircol.AddEqualTimeIntervalsConstraints()

    initial_state = (0.0, 0.0, 0.0, 0.0)
    prog.AddBoundingBoxConstraint(initial_state, initial_state, dircol.initial_state())
    # More elegant version is blocked by drake #8315:
    # prog.AddLinearConstraint(dircol.initial_state() == initial_state)

    final_state = (0.0, np.pi, 0.0, 0.0)
    prog.AddBoundingBoxConstraint(final_state, final_state, dircol.final_state())
    # prog.AddLinearConstraint(dircol.final_state() == final_state)

    R = 10  # Cost on input "effort".
    u = dircol.input()
    dircol.AddRunningCost(R * u[0] ** 2)

    # Add a final cost equal to the total duration.
    dircol.AddFinalCost(dircol.time())

    initial_x_trajectory = PiecewisePolynomial.FirstOrderHold(
        [0.0, 4.0], np.column_stack((initial_state, final_state))
    )  # yapf: disable
    dircol.SetInitialTrajectory(PiecewisePolynomial(), initial_x_trajectory)

    result = Solve(prog)
    assert result.is_success()

    fig, ax = plt.subplots()

    u_trajectory = dircol.ReconstructInputTrajectory(result)
    times = np.linspace(u_trajectory.start_time(), u_trajectory.end_time(), 100)
    u_values = u_trajectory.vector_values(times)

    ax.plot(times, u_values.T)
    ax.set_xlabel("time (seconds)")
    ax.set_ylabel("force (Newtons)")
    display(plt.show())

    # Animate the results.
    x_trajectory = dircol.ReconstructStateTrajectory(result)

    # Make a diagram that includes a visualizer.
    builder = RobotDiagramBuilder(time_step=0.0)
    ConfigureParser(builder.parser())
    builder.parser().AddModelsFromUrl("package://underactuated/models/cartpole.urdf")
    visualizer = ConnectPlanarSceneGraphVisualizer(
        builder.builder(),
        builder.scene_graph(),
        xlim=[-2, 2],
        ylim=[-1.25, 2],
        show=False,
    )
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()

    ani = AnimatePositionTrajectory(x_trajectory, context, diagram.plant(), visualizer)
    display(HTML(ani.to_jshtml()))

    return x_trajectory, u_trajectory


x_trajectory, u_trajectory = dircol_cartpole()

In [ ]:
def open_loop_simulation(x_traj, u_traj):
    robot_builder = RobotDiagramBuilder(time_step=0.0)
    builder = robot_builder.builder()
    parser = robot_builder.parser()
    plant = robot_builder.plant()
    scene_graph = robot_builder.scene_graph()
    ConfigureParser(parser)
    parser.AddModelsFromUrl("package://underactuated/models/cartpole.urdf")
    plant.Finalize()

    traj = builder.AddSystem(TrajectorySource(u_traj))
    builder.Connect(traj.get_output_port(), plant.get_actuation_input_port())

    visualizer = ConnectPlanarSceneGraphVisualizer(
        builder, scene_graph, xlim=[-2, 2], ylim=[-1.25, 2], show=False
    )
    diagram = robot_builder.Build()

    simulator = Simulator(diagram)

    # Simulate and animate
    AdvanceToAndVisualize(
        simulator,
        visualizer,
        u_traj.end_time() if running_as_notebook else 0.1,
    )


open_loop_simulation(x_trajectory, u_trajectory)

In [ ]:
def finite_horizon_lqr(x_trajectory, u_trajectory):
    options = FiniteHorizonLinearQuadraticRegulatorOptions()
    options.x0 = x_trajectory
    options.u0 = u_trajectory

    robot_builder = RobotDiagramBuilder(time_step=0.0)
    builder = robot_builder.builder()
    parser = robot_builder.parser()
    plant = robot_builder.plant()
    scene_graph = robot_builder.scene_graph()
    ConfigureParser(parser)
    parser.AddModelsFromUrl("package://underactuated/models/cartpole.urdf")
    plant.Finalize()

    context = plant.CreateDefaultContext()
    Q = np.diag([10.0, 10.0, 1.0, 1.0])
    options.Qf = Q
    options.input_port_index = plant.get_actuation_input_port().get_index()
    regulator = builder.AddSystem(
        MakeFiniteHorizonLinearQuadraticRegulator(
            plant,
            context,
            t0=options.u0.start_time(),
            tf=options.u0.end_time(),
            Q=Q,
            R=np.eye(1),
            options=options,
        )
    )
    builder.Connect(regulator.get_output_port(0), plant.get_actuation_input_port())
    builder.Connect(plant.get_state_output_port(), regulator.get_input_port(0))
    input_logger = LogVectorOutput(regulator.get_output_port(0), builder)
    state_logger = LogVectorOutput(plant.get_state_output_port(), builder)

    visualizer = ConnectPlanarSceneGraphVisualizer(
        builder, scene_graph, xlim=[-2, 2], ylim=[-1.25, 2], show=False
    )
    diagram = builder.Build()
    simulator = Simulator(diagram)

    # Simulate and animate
    AdvanceToAndVisualize(
        simulator,
        visualizer,
        options.u0.end_time() if running_as_notebook else 0.1,
    )

    fig, ax = plt.subplots(2, 1)
    ax[0].plot(
        options.u0.get_segment_times(),
        options.u0.vector_values(options.u0.get_segment_times()).T,
    )
    input_log = input_logger.FindLog(simulator.get_context())
    ax[0].plot(input_log.sample_times(), input_log.data().T)
    ax[0].legend(("u nominal", "u actual"))

    ax[1].plot(
        options.x0.get_segment_times(),
        options.x0.vector_values(options.x0.get_segment_times()).T,
        "b",
    )
    state_log = state_logger.FindLog(simulator.get_context())
    ax[1].plot(state_log.sample_times(), state_log.data().T, "g")
    ax[1].legend(("x nominal", "x actual"))
    display(plt.show())


finite_horizon_lqr(x_trajectory, u_trajectory)

In [ ]:
def plot_finite_horizon_lqr_results(x_trajectory, u_trajectory):
    options = FiniteHorizonLinearQuadraticRegulatorOptions()
    options.x0 = x_trajectory
    options.u0 = u_trajectory

    plant = MultibodyPlant(time_step=0.0)
    scene_graph = SceneGraph()
    plant.RegisterAsSourceForSceneGraph(scene_graph)
    parser = Parser(plant)
    ConfigureParser(parser)
    parser.AddModelsFromUrl("package://underactuated/models/cartpole.urdf")
    plant.Finalize()

    context = plant.CreateDefaultContext()
    Q = np.diag([10.0, 10.0, 1.0, 1.0])
    options.Qf = Q
    options.input_port_index = plant.get_actuation_input_port().get_index()
    results = FiniteHorizonLinearQuadraticRegulator(
        plant,
        context,
        t0=options.u0.start_time(),
        tf=options.u0.end_time(),
        Q=Q,
        R=np.eye(1),
        options=options,
    )

    lambda_max_S = [
        np.amax(np.linalg.eigvals(results.S.value(t)))
        for t in results.S.get_segment_times()
    ]

    fig, ax = plt.subplots(2, 1)
    ax[0].plot(
        results.S.get_segment_times(),
        lambda_max_S,
    )
    ax[0].set_xlabel("time (s)")
    ax[0].set_ylabel("lambda_max(S(t))")

    ax[1].plot(
        options.x0.get_segment_times(),
        options.x0.vector_values(options.x0.get_segment_times())[1, :].T,
        "b",
    )
    ax[1].plot(
        [options.x0.start_time(), options.x0.end_time()],
        [np.pi / 2, np.pi / 2],
        "--",
        linewidth=1,
    )
    ax[1].set_xlabel("time (s)")
    ax[1].set_ylabel("theta_pend(t)")
    display(plt.show())


plot_finite_horizon_lqr_results(x_trajectory, u_trajectory)

When we plot the largest singular value of $S(t)$ we see that it becomes large around the same time as $\theta_{pend}(t)$ approaches $\frac{\pi}{2}$.  Can you figure out why?